####Setup Unity Catalog
1. Create Workspace
2. Create Data Storage Layer
    1. Storage Account
    2. Storage Container
    3. Databricks Access Connector
    4. Blog Storage Contributor role for Access Conector
3. Setup Unity Catalog Metastore
4. Setup Users and Permissions

#####2.2. Create a storage container for unity catalog metastore (Ex metastore-root)
* Go to your Azure storage account page
* Click "Containers" from the left menu
* Click "+ Container" to create a new container

#####2.3 Create Databricks Access Connector
* Click "+ Create a resource" button on your Azure portal
* Search "access connector for azure databricks"
* Click "Create" button
* Select your Azure subscription and a resource group on the create page
* Give a name to your connector (Ex databricks_access_connector)
* Select a region. Make sure to select the same region as your Databricks workspace and Azure storage account
* Click "Review + Create"
* Click "Create" button after reviewing your settings
* Go to Access Connector Resource page and copy the Resource ID. You will need it later

#####2.4 Grant access to Access connector for your Azure storage account
* Go to your storage account page on your Azure portal
* Click "Access control (IAM)" from the left side memu
* Click "+ Add" link from the top menu and select "Add role assignment"
* Search for "Storage blob data contributor" role and select it
* Click "Next" button
* Choose "Managed idendity" radio button
* Click "+ Select members" on the members page
* Choose your Azure subscription
* Choose "All system managed identities"
* Select your Databricks access connector (Ex databricks_access_connector)
* Click "Select" button
* Click "Review + assign" twice

#####3. Setup Unity Catalog Metastore
* Go to Databricks Account console
    * Click your name at the top right corner of the Databricks workspace
    * Click Manage Accounts from the menu
    * Databricks account console will open in a new browser tab
* Perform following in the Databricks Account Console
    * Click Data from the left side menu
    * Click "Create Metastore" button
    * Give a name for metastore (Ex - scholarnest-meta)
    * Choose a region for metatore deployment. Make sure to choose the same region as your workspace and storage account (Ex eastus)
    * Type storage container path for metastore storage (Ex metastore-root@prashantsa.dfs.core.windows.net/)
    * Paste the Access connector resource id
    * Clcik the "Create" button
    * Select all workspace names to connect it with the metastore
    * Click "Assign" button

#####4. Provision your company users to Unity Catalog
1. Get SCIM connection details
    1. Click settings menu from the left side of Databricks Account Console
    2. Click "Setup user provisioning" button
    3. Copy SCIM token
    4. Copy SCIM URL
    5. Click "Done" button
2. Sync corporate users from your Active Directory
    1. Go to your organizations Active Directory (Ex Your Azure Active Directory)
    2. Select "Enterprise Applications" from the left side menu
    3. Clcik "+ New Application"
    4. Search for "Azure Databricks SCIM Provisioning Connector" and select it
    5. Click "Create" button
    6. Clcik "Provisioning" from the left side menu on the SCIM Provisioning Connector page
    7. Click "Get Started" button
    8. Select Provisioning mode (Ex Automatic)
    9. Paste Databricks Account SCIM URL that you copied earlier
    10. Paste Databricks Account SCIM token that you copied earlier
    11. Click "Test Connection" to confirm the connectivity
    12. Click "Save" link at the top
    13. Go back to "SCIM Provisioning Connector" page
    14. Click "Users and groups" from the left side menu
    15. Click "+ Add user/group"
    16. Click "Not Selected" and select desired users
    17. Click "Assign" button
    18. Click "Provisioning" from the left side menu
    19. Click "Start Provisioning" button from the top
    20. Wait for "Initial cycle completed." message

#####5. Create user group
1. Click "User management" from the left side of Databricks Account Console
2. Go to groups tab
3. Click Add group button
4. Type group name (Ex scholarnest-dev)
5. Click Save button
6. Click "Add members" button to add new members to the group

#####1. Create a DEV and a QA catalog.

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS dev;

#####2. Create a demo_db database in DEV catalog.

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS dev.demo_db;

DROP DATABASE IF EXISTS qa.default;

%md
#####4. Grant following privilages to uc-dev-group
1. USE CATALOG on DEV
2. ALL PRIVILEGES on demo_db

In [0]:
%sql
GRANT USE CATALOG ON CATALOG dev TO `uc-dev-group`;
GRANT ALL PRIVILEGES ON DATABASE dev.demo_db to `uc-dev-group`;

relevant spark command 
%python
spark.sql("""
    GRANT USE CATALOG ON CATALOG dev TO `uc-dev-group`;
    GRANT ALL PRIVILEGES ON DATABASE dev.demo_db to `uc-dev-group`;
""")

#####5. Create volume under demo_db

In [0]:
%sql
CREATE VOLUME dev.demo_db.files;

In [0]:
%fs ls /Volumes/dev/demo_db/files/

#####9. Create an external location to an existing container/directory in ADLS (Use the Catalog explorer)

#####10. Describe external location

In [0]:
%sql
DESCRIBE EXTERNAL LOCATION `el-storage-dbfs-container`;

#####11. Grant READ FILES privilage on external location to uc-dev-group

In [0]:
%sql
GRANT READ FILES ON EXTERNAL LOCATION `el-storage-dbfs-container` TO `uc-dev-group`

#####12. List content of the external location

In [0]:
%fs ls abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net/

###### Create Delta table. You can also create a Delta table from an existing data source using the CREATE TABLE AS SELECT or CREATE TABLE USING commands. For example, you can create a Delta table from a CSV file with the following command:

Below syntax to create an external table

In [0]:
%sql
USE CATALOG dev;
USE SCHEMA demo_db;
CREATE TABLE people
USING delta
LOCATION 'abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net/people/'
AS SELECT *
FROM json.`abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net/people.json`

In [0]:
%sql
select * from dev.demo_db.people

##### below syntax to create managed delta table

In [0]:
%sql
USE CATALOG dev;
USE SCHEMA demo_db;
CREATE TABLE people01
USING delta
AS SELECT *
FROM json.`abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net/people.json`

####### Using DataFrame API

Another way to create a Delta table is to use the DataFrame API in Python, R, Scala, or Java. You can use this API to create a DataFrame from various sources, such as files, databases, or Spark tables, and then write it to a Delta table using the write or save methods. For example, you can create a Delta table from a Parquet file with the following Python code:

In [0]:
%fs ls /mnt

In [0]:
%fs ls /mnt/u01

##### in the below code we are creating a delta table using the json file,delta table will not be listed in unity catalog. to list this table in unity catalog. We need to create unity catalog table using the delta table.

In [0]:
%python
# Read the json file into a DataFrame
df = spark.read.json('/mnt/u01/flight-time.json')
# Write the DataFrame to a Delta table
df.write.format('delta').save('/mnt/u01/flight-time-delta')

In [0]:
%sql
USE CATALOG dev;
USE SCHEMA demo_db;
CREATE TABLE flight_time
USING delta
AS SELECT *
FROM delta.`abfss://dbfs-container@adlssmiloeabronzedev001.dfs.core.windows.net/flight-time-delta/`

#### using this link for creating delta table in different format
https://dezimaldata.medium.com/5-ways-to-create-a-spark-delta-table-in-databricks-c53af3016660

In [0]:
%fs ls /mnt/u01

In [0]:
base_dir = "/mnt/u01"
flight_schema_ddl = """FL_DATE DATE, OP_CARRIER STRING, OP_CARRIER_FL_NUM INT, ORIGIN STRING, 
          ORIGIN_CITY_NAME STRING, DEST STRING, DEST_CITY_NAME STRING, CRS_DEP_TIME INT, DEP_TIME INT, 
          WHEELS_ON INT, TAXI_IN INT, CRS_ARR_TIME INT, ARR_TIME INT, CANCELLED STRING, DISTANCE INT"""

flight_time_df = (spark.read.format("json")
                    .schema(flight_schema_ddl)
                    .option("dateFormat", "M/d/y")
                    .load(f"{base_dir}/flight-time.json")
)

#####1. Create a delta table uing Spark SQL

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dev.demo_db.flight_time_tbl (
    FL_DATE DATE, 
    OP_CARRIER STRING, 
    OP_CARRIER_FL_NUM INT, 
    ORIGIN STRING, 
    ORIGIN_CITY_NAME STRING, 
    DEST STRING, 
    DEST_CITY_NAME STRING, 
    CRS_DEP_TIME INT, 
    DEP_TIME INT, 
    WHEELS_ON INT, 
    TAXI_IN INT, 
    CRS_ARR_TIME INT, 
    ARR_TIME INT, 
    CANCELLED STRING, 
    DISTANCE INT
) USING DELTA

#####2. Load data into delta table

In [0]:
flight_time_df.write.format("delta").mode("append").saveAsTable("dev.demo_db.flight_time_tbl")


In [0]:
%sql
select * from dev.demo_db.flight_time_tbl